In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MLExp').getOrCreate()

In [3]:
training = spark.read.csv('test3.csv', inferSchema=True, header=True)

In [4]:
training.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Gagan| 25|         5| 40000|
| Ajay| 28|         7| 45000|
|Sunny| 30|        10| 48000|
|Riyan| 15|         1| 10000|
|Radha| 32|        10| 45000|
|Rishi| 38|        17| 80000|
+-----+---+----------+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [7]:
from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols=['Age', 'Experience'], outputCol='IndependentFeature')

In [8]:
output = featureAssembler.transform(training)

In [9]:
output.show()

+-----+---+----------+------+------------------+
| Name|Age|Experience|Salary|IndependentFeature|
+-----+---+----------+------+------------------+
|Gagan| 25|         5| 40000|        [25.0,5.0]|
| Ajay| 28|         7| 45000|        [28.0,7.0]|
|Sunny| 30|        10| 48000|       [30.0,10.0]|
|Riyan| 15|         1| 10000|        [15.0,1.0]|
|Radha| 32|        10| 45000|       [32.0,10.0]|
|Rishi| 38|        17| 80000|       [38.0,17.0]|
+-----+---+----------+------+------------------+



In [10]:
finalized_data = output.select('IndependentFeature', 'Salary')

In [11]:
finalized_data.show()

+------------------+------+
|IndependentFeature|Salary|
+------------------+------+
|        [25.0,5.0]| 40000|
|        [28.0,7.0]| 45000|
|       [30.0,10.0]| 48000|
|        [15.0,1.0]| 10000|
|       [32.0,10.0]| 45000|
|       [38.0,17.0]| 80000|
+------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='IndependentFeature', labelCol='Salary')
regressor = regressor.fit(train_data)

In [13]:
regressor.coefficients

DenseVector([1306.6826, 2196.367])

In [14]:
regressor.intercept

-9635.375232033026

In [15]:
pred_results = regressor.evaluate(test_data)

In [16]:
pred_results.predictions.show()

+------------------+------+-----------------+
|IndependentFeature|Salary|       prediction|
+------------------+------+-----------------+
|       [30.0,10.0]| 48000|51528.77220896316|
+------------------+------+-----------------+



In [17]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(3528.772208963157, 12452233.302750718)